<a href="https://colab.research.google.com/github/skyfuryonline/semeval/blob/master/%E4%BF%AE%E6%94%B9%E6%95%B0%E6%8D%AE%E6%A0%BC%E5%BC%8F%E6%B5%8B%E8%AF%95opus_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [38]:
from datasets import Dataset
from transformers import AutoTokenizer

# 示例数据
# data = {
#         "source_locale": ["en","en"],
#         "target_locale": ["fr","fr"],
#         "source": ["How many Star Wars movies has Chewbacca appeared in?","'Good!' said Norbert, 'there's one of my father's spies gone; that leaves only the little cripple Napier.'"],
#         "target": ["Combien de films Star Wars Chewbacca est-il apparu?",'– Bon! dit Norbert, voilà un des espions de mon père parti ; il ne reste plus que le petit boiteux Napier.'],
# }
from datasets import load_dataset

data = load_dataset("opus_books", "en-fr")
data = data['train'].train_test_split(test_size=0.2)

train_data = {
    "source_locale": [],
    "target_locale": [],
    "source":[],
    "target":[],
}
test_data = {
    "source_locale": [],
    "target_locale": [],
    "source":[],
    "target":[],
}

cnt = 1
for item in data['train']:
  for lang,sentence in item['translation'].items():
    if cnt %2 != 0:
      train_data['source_locale'].append(lang)
      train_data['source'].append(sentence)
    else:
      train_data['target_locale'].append(lang)
      train_data['target'].append(sentence)
    cnt += 1

cnt = 1
for item in data['test']:
  for lang,sentence in item['translation'].items():
    if cnt %2 != 0:
      test_data['source_locale'].append(lang)
      test_data['source'].append(sentence)
    else:
      test_data['target_locale'].append(lang)
      test_data['target'].append(sentence)
    cnt += 1



In [37]:
# 创建Dataset对象
train_data = Dataset.from_dict(train_data)
test_data = Dataset.from_dict(test_data)
print(train_data[0])
print(test_data[0])

NameError: name 'test_data' is not defined

In [ ]:
column_name = dataset.column_names

# 加载 T5 模型的分词器
model_id = "google-t5/t5-small"  # 使用 T5-small 模型
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [3]:
# 任务说明和输入格式
prefix = "translate English to French: "

# 预处理函数
def preprocess_function(examples):
    # 为每个句子添加任务描述前缀
    inputs = [prefix + example for example in examples['source']]
    targets = [example for example in examples['target']]
    # 使用 T5 分词器对输入和目标进行编码
    model_inputs = tokenizer(inputs, text_target=targets, max_length=64, truncation=True, padding="max_length")
    return model_inputs

# 预处理数据
tokenized_data = dataset.map(preprocess_function, batched=True,remove_columns=column_name)

# 查看预处理后的数据
print(tokenized_data[0])

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

{'input_ids': [13959, 1566, 12, 2379, 10, 571, 186, 2042, 10265, 4876, 65, 2556, 210, 9305, 658, 4283, 16, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [2570, 4232, 29, 20, 4852, 2042, 10265, 2556, 210, 9305, 658, 259, 18, 173, 3, 10198, 76, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [7]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_id)

In [8]:
! pip install evaluate
! pip install sacrebleu
import evaluate
metric = evaluate.load("sacrebleu")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.3 MB/s eta 0:00:00


In [9]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [10]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    # save_total_limit=3,
    num_train_epochs=3,
    # predict_with_generate=True,
    fp16=True,
    # push_to_hub=True,
    report_to ="none",
    gradient_checkpointing=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-a5906008e70a>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss,Validation Loss


TrainOutput(global_step=20, training_loss=1.1629796981811524, metrics={'train_runtime': 8.4264, 'train_samples_per_second': 4.747, 'train_steps_per_second': 2.374, 'total_flos': 676709007360.0, 'train_loss': 1.1629796981811524, 'epoch': 20.0})

In [14]:
text = "translate English to French: hello"

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)
inputs = tokenizer(text, return_tensors="pt").input_ids
inputs = inputs.to("cuda")

In [21]:
from transformers import AutoModelForSeq2SeqLM
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'de la mer'